<a href="https://colab.research.google.com/github/smshozab/FieldMatrix/blob/main/Agriculture_Research_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install segmentation-models-pytorch


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 KB 430.8 kB/s eta 0:00:001m451.3 kB/s eta 0:00:01
  Using cached torchvision-0.18.1-cp310-cp310-manylinux1_x86_64.whl (7.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
  Using cached torch-2.3.1-cp310-cp310-manylinux1_x86_64.whl (779.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 KB 2.2 MB/s eta 0:00:002.2 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.5 MB/s eta 0:00:001.5 MB/s eta 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:02
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none

In [ ]:
import cv2
import numpy as np
import torch
from torchvision import transforms
import segmentation_models_pytorch as smp

# Load the segmentation model (e.g., U-Net with a ResNet backbone)
model = smp.Unet(encoder_name='resnet34', encoder_weights='imagenet', in_channels=3, classes=1)
model.eval()  # Set model to evaluation mode

# Function to calculate GSD (Ground Sampling Distance)
def calculate_gsd(drone_height, sensor_width, image_width, focal_length):
    return (drone_height * sensor_width) / (focal_length * image_width)

# Function to calculate real-world distances
def calculate_distance(px_distance, gsd):
    return px_distance * gsd

# Load and preprocess the image
image_path = '/home/cheeta/Downloads/sample/p5.jpeg'
image = cv2.imread(image_path)
image_resized = cv2.resize(image, (256, 256))  # Resize for the model

# Transform image for the model
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image_input = transform(image_resized).unsqueeze(0)  # Add batch dimension

# Predict the segmentation mask
with torch.no_grad():
    mask = model(image_input)[0].squeeze().cpu().numpy()
mask = (mask > 0.5).astype(np.uint8)  # Threshold the mask

# Find contours of the segmented regions
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Calculate centroids
centroids = []
for contour in contours:
    M = cv2.moments(contour)
    if M['m00'] != 0:
        centroid_x = int(M['m10'] / M['m00'])
        centroid_y = int(M['m01'] / M['m00'])
        centroids.append((centroid_x, centroid_y))

# Parameters (replace with actual values)
drone_height = 200  # Height from which the image was taken (meters)
sensor_width = 12  # Width of the camera sensor (mm)
image_width = image.shape[1]  # Width of the image in pixels
focal_length = 8  # Focal length of the camera (mm)

# Calculate GSD
gsd = calculate_gsd(drone_height, sensor_width, image_width, focal_length)

# Calculate distances between plants
distances = []
for i, (cx1, cy1) in enumerate(centroids):
    for j, (cx2, cy2) in enumerate(centroids[i+1:]):
        px_distance = np.sqrt((cx2 - cx1)**2 + (cy2 - cy1)**2)
        real_distance = calculate_distance(px_distance, gsd)
        distances.append((i, i + j + 1, real_distance))

# Output the distances
for (i, j, distance) in distances:
    print(f"Distance between plant {i} and plant {j}: {distance:.2f} meters")
    # cv2.line(image, centroids[i], centroids[j], (0, 255, 0), 2)
    # mid_point = ((centroids[i][0] + centroids[j][0]) // 2, (centroids[i][1] + centroids[j][1]) // 2)
    # cv2.putText(image, f"{distance:.2f}m", mid_point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# # Save the segmented mask as an image
# segmented_image_path = 'segmented_image.png'
# cv2.imwrite(segmented_image_path, mask_resized * 255)  # Save as binary mask image


# Display the image with detections (optional)
for centroid in centroids:
    cv2.circle(image, (int(centroid[0]), int(centroid[1])), 5, (0, 255, 0), -1)
cv2.imshow('Segmented Plants', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'segmentation_models_pytorch'